In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, plot_confusion_matrix, f1_score


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# EDA
df = pd.read_csv('/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip')
print("Shape of data: " + str(df.shape))
print("\n\nNull's in data:")
print(sum(df.isnull().sum()))
print("\n\nDescribing data:")
print(df.describe())
le = LabelEncoder()
df['type'] = le.fit_transform(df['type'])
df = df.drop(['id'], axis = 1)
plt.hist(df['type'], bins='auto', color='#0504aa', alpha=0.7, rwidth=0.85)

# Scatter plots of each pair of features
sns.pairplot(data = df)
plt.show()

plt.figure(figsize=(40,20))
sns.boxplot(data=df)


# Let`s see the correlation between all variables
plt.figure(figsize=(12,10))
cor = df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

Classes are almost balanced. Outliers there aren't here. Dataset is small, so to build very good model will be difficult, but let's try

**The best model**

Here is ensemble of three models which votes how classificate new point.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans

y = df['type']
X = df.drop(['type', 'color'], axis = 1)
pol = PolynomialFeatures()
t = pol.fit_transform(X)
X_new = pd.DataFrame(t)
X_new = np.log1p(X_new)
X_train, X_test, y_train, y_test = train_test_split(X_new, y, random_state=0)

# Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy Score is: " + str(accuracy_score(y_pred, y_test)))
print("\n\n")
prob_1 = clf.predict_proba(X_test)


# NNC
clf_2 = MLPClassifier(random_state = 1, hidden_layer_sizes = 20, activation = 'relu', max_iter = 5000, alpha = 0.000001, shuffle = True)
clf_2.fit(X_train, y_train)
y_pred = clf_2.predict(X_test)
print("Accuracy Score is: " + str(accuracy_score(y_pred, y_test)))
print("\n\n")
prob_2 = clf_2.predict_proba(X_test)


# LogisticRegression
from sklearn.linear_model import LogisticRegression
lg3 = LogisticRegression(random_state = 13, n_jobs = -1)
lg3.fit(X_train, y_train)
y_pred = lg3.predict(X_test)
print("Accuracy Score is: " + str(accuracy_score(y_pred, y_test)))
print("\n\n")
prob_3 = lg3.predict_proba(X_test)




def predict(prob_1, prob_2, prob_3):
    numb_of_samples = prob_1.shape[0]
    label_ = []
    for i in range(numb_of_samples):
        indx_ = [np.where(prob_1[i] == max(prob_1[i]))[0][0], np.where(prob_2[i] == max(prob_2[i]))[0][0], np.where(prob_3[i] == max(prob_3[i]))[0][0]]
        prob_ = [max(prob_1[i]), max(prob_2[i]), max(prob_3[i])]
        if len(np.unique(indx_)) <= 2:
            label_.append(max(set(indx_), key=indx_.count))
        else:
            label_.append(np.where(prob_ == max(prob_))[0][0])
    return label_
        
        
y_pred = predict(prob_1, prob_2, prob_3)
print("Accuracy Score of Ensembe is: " + str(accuracy_score(y_pred, y_test)))



df_test = pd.read_csv('/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip')
X_sub_test = df_test.drop(['id', 'color'], axis = 1)
t = pol.transform(X_sub_test)
X_sub_test = pd.DataFrame(t)
X_sub_test = np.log1p(X_sub_test)

prob_1 = clf.predict_proba(X_sub_test)
prob_2 = clf_2.predict_proba(X_sub_test)
prob_3 = lg3.predict_proba(X_sub_test)

y_sub_pred = predict(prob_1, prob_2, prob_3)

submission = pd.DataFrame(le.inverse_transform(y_sub_pred), index = df_test['id'], columns = ['type'])
print(submission)
import os
print(os.getcwd())
submission.to_csv('C://sub_nn.csv', header = True)
